In [1]:
import io
import PIL.Image
from torchvision.transforms import ToTensor
    
def gen_image():
    buf = io.BytesIO()
    plt.savefig(buf, format='jpeg')
    buf.seek(0)
    image = PIL.Image.open(buf)
    image = ToTensor()(image).unsqueeze(0)
    return image


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tensorboardX import SummaryWriter

In [18]:
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.fc = nn.Linear(1, 1)
    
    def forward(self, x):
        return self.fc(x)
net = MyNet() 
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

In [19]:
train_x = torch.linspace(-2, 5, 50).view(-1, 1)
train_y = train_x * 2 + 3

In [20]:
writer = SummaryWriter()
for iter_no in range(200):
    optimizer.zero_grad()
    output = net(train_x)
    loss = criterion(output, train_y)
    
    for name, param in net.named_parameters():
        if name == "fc.bias":
            writer.add_scalar('data/fc.bias', param.item(), iter_no)
        elif name == "fc.weight":
            writer.add_scalar('data/fc.weight', param.item(), iter_no)
    writer.add_scalar('data/loss', loss.item(), iter_no)
    
    loss.backward()
    optimizer.step()    